# CODICE PROGETTO 32 OPTIMIZATION 


### Implementazione algoritmo risolutivo

In [ ]:
import numpy as np
import random
import time
from scipy.optimize import *

# !pip install pyMCFSimplex
from pyMCFSimplex import *

#### 1. Estrazione dei dati dai file .dmx e .qfc utilizzati per la generazione delle istanze del problema
La prima funzione *matrice_Q(nome_file)* serve per estrarre i costi quadratici che si trovano nella diagonale di Q,

mentre la seconda funzione *leggi_file_dimacs(nome_file)* serve per estrarre le seguenti quantità: 
 - u, b, q
 - numero nodi
 - numero archi 

In [ ]:
def matrice_Q(nome_file):
    vettore = []
    with open(nome_file, 'r') as file:
        dimensione = int(file.readline()) # la prima riga è la dimensione del vettore
        valori = file.readline().split() # dalla seconda riga 
        vettore = [float(valore) for valore in valori]
        
        #error check 
        if len(vettore) != dimensione:
            raise ValueError("Il numero di valori nel file non corrisponde alla dimensione specificata")
    
    dimensione = len(vettore)
    matrice = np.zeros((dimensione, dimensione)) # creazione matrice 
    np.fill_diagonal(matrice, vettore)  # si riempie diagonale
    
    return matrice, vettore


def leggi_file_dimacs(nome_file):
    numero_nodi = 0
    numero_archi = 0
    u = []
    b = []
    q = []
    from_=[]
    to_=[]
    edges = []

    with open(nome_file, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) > 0:
                if parts[0] == 'p':
                    # legge il numero di nodi e archi dal problema
                    numero_nodi = int(parts[2])
                    numero_archi = int(parts[3])
                    # inizializza il vettore di supply con zeri
                    b = [0] * numero_nodi
                elif parts[0] == 'n':
                    # legge i valori di supply per i nodi
                    nodo_id = int(parts[1])
                    supply = int(parts[2])
                    # assegna il valore di supply al nodo corrispondente
                    b[nodo_id - 1] = supply
                elif parts[0] == 'a':
                    # leggi l'arco e il suo cotso
                    from_node = int(parts[1])
                    to_node = int(parts[2])
                    max_capacity = int(parts[4])
                    costo = int(parts[5])  # leggiamo costo corretto
                    from_.append(from_node)
                    to_.append(to_node)
                    u.append(max_capacity)
                    q.append(costo)
                    edges.append((from_node , to_node ))

    return numero_nodi, numero_archi, u, b, q, edges,from_, to_

In [ ]:
# esempio utilizzo 
nome_file = "1000/netgen-1000-1-1-a-a-s.qfc"
Q_d = matrice_Q(nome_file)[1]
Q = matrice_Q(nome_file)[0]

# Esempio di utilizzo
nome_file = '1000/netgen-1000-1-1-a-a-s.dmx'  
numero_nodi, numero_archi, u, b, q, edges,from_ , to_ = leggi_file_dimacs(nome_file)

# creazione della matrice E 
E = np.zeros((numero_nodi, len(edges)), int)

# import pandas as pd
# E = pd.DataFrame(E)
# metto nomi delle colonne delle matrice E uguali agli archi
# E.columns = [edges]

#### 2. Algoritmo

Step: 

0. Inizializzazione di x<sub>0</sub> con generazione casuale soggetta al vincolo 0 &le; x<sub>0</sub> &le; u
1. Calcolo del gradiente, risoluzione del sottoproblema lineare usando il solver pyMCFSimplex (x̄), determinazione della direzione
2. Determinazione dello step size &alpha;
3. Aggiornamento della posizione e check terminazione sul prodotto scalare (<grad, d>)

In [ ]:
# Questa funzione crea un nuovo file .dmx in cui i valori delle righe che iniziano con 'a' (definizione archi)
# vengono sostituiti con i valori presenti nel vettore gradient. L'obiettivo è modificare il file di input 
# (input_file) in modo da utilizzare al posto dei costi lineari del file originale, il gradiente 
# (vogliamo risolvere il sottoproblema lineare, aka l'approssimazione lineare del problema Taylor 1st ordine).   

def modify_file_with_gradient(input_file, output_file, gradient):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    # inizalizzazione dell'indice per il vettore gradient
    gradient_index = 0

    # modifica delle righe che iniziano con 'a'
    for i in range(len(lines)):
        if lines[i].startswith('a'):
            words = lines[i].split()
            words[-1] = str(gradient[gradient_index])
            gradient_index += 1
            lines[i] = ' '.join(words)

    # creazione nuovo file
    with open(output_file, 'w') as file:
        file.writelines(lines)

In [ ]:

k = 0
epsilon = 0.01
max_iter = 1000
start_time = time.time()
# prodotto_scalare = float("inf")

# STEP 0 : inizializzazione di x_0
seed = 42
np.random.seed(seed)
x_old = []

for u_i in u:
    x_i = random.randint(0, u_i)
    x_old.append(x_i)


while k <= max_iter or prodotto_scalare >= epsilon:
    
    # STEP 1 : calcolo gradiente
    gradient = (2 * np.dot(Q, x_old)) + q
    gradient = gradient.tolist()
    modify_file_with_gradient('1000/netgen-1000-1-1-a-a-s.dmx', 'output3.dmx', gradient)

    # risoluzione del problema lineare (ricerca di argmin) con MCF solver
    FILENAME = 'output3.dmx'
    f = open(FILENAME,'r')
    inputStr = f.read()
    f.close()
    mcf = MCFSimplex()
    mcf.LoadDMX(inputStr)
    mcf.SolveMCF()
    if mcf.MCFGetStatus() == 0:
        if k % 100 == 0:
            print( 'step size: ', alpha, 'iterazione: ' ,k," Optimal solution: %s" %mcf.MCFGetFO())
    else:
        print( "Problem unfeasible!")
    vettore_soluzione = {}   
    sol_x = [0] * numero_archi
    for key in vettore_soluzione:
        if key <= 1000:

            sol_x[key-1] = vettore_soluzione[key]

    # calcolo delle x_bar e determinazione della direzione di ricerca
    x_bar = sol_x
    d = [a - b for a, b in zip(x_bar, x_old)] # direzione



    # STEP 2 : determinazione alpha, step size
    alpha= 2 / (2+k)
   
    # STEP 3 : aggiornamento della posizione
    x_new = []
    for i in range(len(x_old)):
        x_new.append(x_old[i] + alpha * d[i])

    # check terminazione 
    gradient_per_check=(2 * np.dot(Q, x_new) ) + q
    prodotto_scalare = np.dot(gradient_per_check, d)
   
    # aggiornamento poszione e incremento numero di iterazioni
    x_old = x_new
    k += 1
    
end_time = time.time()
tempo_tot = end_time - start_time

print('prodotto scalare: ' ,prodotto_scalare, 'step size: ' ,alpha, 'iterazioni:', k)
print('tempo totale' , tempo_tot)

In [ ]:
def algoritmo(nome_file, epsilon, max_iter, Q, q, u, d, gradient, numero_archi, sol_x):
    k = 0 

    start_time = time.time()
    # prodotto_scalare = float("inf")

    # STEP 0 : inizializzazione di x_0
    seed = 42
    np.random.seed(seed)
    x_old = []

    for u_i in u:
        x_i = random.randint(0, u_i)
        x_old.append(x_i)


    while k <= max_iter or prodotto_scalare >= epsilon:
        
        # STEP 1 : calcolo gradiente
        gradient = (2 * np.dot(Q, x_old)) + q
        gradient = gradient.tolist()
        modify_file_with_gradient('1000/netgen-1000-1-1-a-a-s.dmx', 'output3.dmx', gradient)

        # risoluzione del problema lineare (ricerca di argmin) con MCF solver
        FILENAME = 'output3.dmx'
        f = open(FILENAME,'r')
        inputStr = f.read()
        f.close()
        mcf = MCFSimplex()
        mcf.LoadDMX(inputStr)
        mcf.SolveMCF()
        if mcf.MCFGetStatus() == 0:
            if k % 100 == 0:
                print( 'step size: ', alpha, 'iterazione: ' ,k," Optimal solution: %s" %mcf.MCFGetFO())
        else:
            print( "Problem unfeasible!")
        vettore_soluzione = {}   
        sol_x = [0] * numero_archi
        for key in vettore_soluzione:
            if key <= 1000:

                sol_x[key-1] = vettore_soluzione[key]

        # calcolo delle x_bar e determinazione della direzione di ricerca
        x_bar = sol_x
        d = [a - b for a, b in zip(x_bar, x_old)] # direzione

        # STEP 2 : determinazione alpha, step size
        alpha= 2 / (2+k)
    
        # STEP 3 : aggiornamento della posizione
        x_new = []
        for i in range(len(x_old)):
            x_new.append(x_old[i] + alpha * d[i])

        # check terminazione 
        gradient_per_check=(2 * np.dot(Q, x_new) ) + q
        prodotto_scalare = np.dot(gradient_per_check, d)
    
        # aggiornamento poszione e incremento numero di iterazioni
        x_old = x_new
        k += 1
        
    end_time = time.time()
    tempo_tot = end_time - start_time

    print('prodotto scalare: ' ,prodotto_scalare, 'step size: ' ,alpha, 'iterazioni:', k)
    print('tempo totale' , tempo_tot)
    return 

In [ ]:
def showModuleFunctionality():
   
    nmx = mcf.MCFnmax()
    mmx = mcf.MCFmmax()
    pn = mcf.MCFnmax()
    pm = mcf.MCFmmax()

    pU = []
    caps = new_darray(mmx)
    mcf.MCFUCaps(caps)
    for i in range(0, mmx):
        pU.append(darray_get(caps, i))

    pC = []
    costs = new_darray(mmx)
    mcf.MCFCosts(costs)
    for i in range(0, mmx):
        pC.append(darray_get(costs, i))

    pDfct = []
    supply = new_darray(nmx)
    mcf.MCFDfcts(supply)
    for i in range(0, nmx):
        pDfct.append(darray_get(supply, i))

    pSn = []
    pEn = []
    startNodes = new_uiarray(mmx)
    endNodes = new_uiarray(mmx)
    mcf.MCFArcs(startNodes, endNodes)
    for i in range(0, mmx):
        pSn.append(uiarray_get(startNodes, i) + 1)
        pEn.append(uiarray_get(endNodes, i) + 1)

    print("arc flow")
    length = mcf.MCFm()
    flow = new_darray(length)
    length = mcf.MCFn()
    nms = new_uiarray(length)
    mcf.MCFGetX(flow, nms)

   

    for i in range(0, length):
        print("flow", darray_get(flow, i), "arc", uiarray_get(nms, i))
        vettore_soluzione[uiarray_get(nms, i)] = darray_get(flow, i)

    print("node potentials")
    length = mcf.MCFn()
    costs = new_darray(length)
    mcf.MCFGetPi(costs, nms)
    for i in range(0, length):
        print("flow", darray_get(costs, i), "node", i + 1)

    print("reading graph - arcs")
    length = mcf.MCFm()
    startNodes = new_uiarray(length)
    endNodes = new_uiarray(length)
    mcf.MCFArcs(startNodes, endNodes)
    for i in range(0, length):
        print("Arc %s: start %s end %s" % (i, uiarray_get(startNodes, i) + 1, uiarray_get(endNodes, i) + 1))

    print("reading graph - costs")
    length = mcf.MCFm()
    costs = new_darray(length)
    mcf.MCFCosts(costs)
    for i in range(0, length):
        print("Arc %s: cost %s" % (i, darray_get(costs, i)))

    print("reading graph - capacities")
    length = mcf.MCFm()
    caps = new_darray(length)
    mcf.MCFUCaps(caps)
    for i in range(0, length):
        print("Arc %s: capacities %s" % (i, darray_get(caps, i)))

    print("reading nodes - supply/demand")
    length = mcf.MCFn()
    supply = new_darray(length)
    mcf.MCFDfcts(supply)
    for i in range(0, length):
        print("Node %s: demand %s" % (i + 1, darray_get(supply, i)))

    return vettore_soluzione  # Restituisci il vettore_soluzione alla fine della funzione

In [ ]:
def mega_f()